In [125]:
from stock_data_ingestion import ingest_stocks_to_df
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display 
from keras.layers import LSTM, Dense, Input, Dropout, Activation, BatchNormalization, LayerNormalization, GRU, Bidirectional
from keras import Sequential
from sklearn.model_selection import TimeSeriesSplit
from keras.optimizers import Adam, SGD
from keras.losses import MeanSquaredError
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.activations import relu
from keras.callbacks import EarlyStopping

## Data Preprocessing

In [103]:
def get_data_targets(stock_arr, sequence_inlength):
    """where stock_arr is sequential np.array and 
    sequence_inlength is the length of the data sequences"""
    x, y = list(), list()
    for index in range(sequence_inlength, len(stock_arr)):
        window = stock_arr[index-sequence_inlength:index]
        x.append(window)
        y.append([stock_arr[index]])
    data = np.array(x)
    targets = np.array(y)
    return data, targets
def test_train_split(data, targets, split=0.8):
    """both of np.array"""
    data_train_len = int(len(data)*split)
    targets_train_len = int(len(targets)*split)
    data_tr, targets_tr = data[:data_train_len], targets[:targets_train_len]
    data_test, targets_test = data[data_train_len:], targets[targets_train_len:]
    return data_tr, targets_tr, data_test, targets_test
def to_ts_data(stock=stock_list[0], sequence_inlength=60):
    stock_arr = stocks_df[stocks_df.company_name == stock]['Adj Close'].to_numpy()
    data, targets = get_data_targets(stock_arr, sequence_inlength)
#     display(data, targets)
    data_tr, targets_tr, data_test, targets_test = test_train_split(data, targets, split=0.8)
    data_gen_train = TimeseriesGenerator(data_tr, targets_tr, length=sequence_inlength)
    data_gen_test = TimeseriesGenerator(data_test, targets_test, length=sequence_inlength)
    ##shape (number of batches, input number of steps before prediction, feature-dimensions for input)
    return data_gen_train, data_gen_test

In [104]:
## comment first line out after first use to prevent 
## giving the source website for the stocks too much traffic. 
# stocks_df = ingest_stocks_to_df()
# stock_list = stocks_df.company_name.unique()
# # displays
# display(stocks_df)
# for stock in stock_list:
#     print(stock, 'len', len(stocks_df[stocks_df.company_name == stock].index))
# sns.lineplot(x=stocks_df.index, y='Adj Close',data=stocks_df, legend='auto', hue='company_name')
# plt.show()
# data_gen_train, data_gen_test = to_ts_data() 


## Model Building

In [136]:
def lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def seq_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def bi_seq_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(Bidirectional(lstm_layer))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def stack_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(lstm_layer)
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def bi_stack_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(Bidirectional(lstm_layer))
    model.add(Bidirectional(lstm_layer))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def bi_back_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    forward_layer = LSTM(nodes_lstm, return_sequences=True)
    backward_layer = LSTM(nodes_lstm, return_sequences=True,
                       go_backwards=True)
    model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def bi_back_stack_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    model = Sequential()
    forward_layer = LSTM(nodes_lstm, return_sequences=True)
    backward_layer = LSTM(nodes_lstm, return_sequences=True, go_backwards=True)
    model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
    model.add(Bidirectional(forward_layer, backward_layer=backward_layer))

    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def stack_3_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(lstm_layer)
    model.add(lstm_layer)
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def deep_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(Dense(32))    
    model.add(Dropout(0.5))
    model.add(Activation(relu))
    model.add(Dense(32))    
    model.add(Dropout(0.5))
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def deep_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
#     model.add(Dropout(0.5))
    model.add(Dense(32))    
    model.add(Dropout(0.5))
    model.add(Activation(relu))
    model.add(Dense(32))    
    model.add(Dropout(0.5))
    model.add(Activation(relu))
    model.add(Dense(32))    
    model.add(Dropout(0.5))
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def lstm_dense_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(Dense(32))
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def batch_norm_lstm_model(nodes_lstm=50, dropout=0.5, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(BatchNormalization())
    model.add(Dense(32))    
    model.add(BatchNormalization())
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def norm_lstm_model(nodes_lstm=50, dropout=0.5, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(LayerNormalization())
    model.add(Dense(32))    
    model.add(LayerNormalization())
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
# lstm_model().summary()
def batch_norm_drop_lstm_model(nodes_lstm=50, dropout=0.5, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(32))    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def gru_model(nodes=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    model = Sequential()
    model.add(GRU(nodes))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model

## model evaluation

In [106]:
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

In [107]:
model = lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 4s 62ms/step - loss: 21.4959 - mse: 21.4959
Epoch 2/100
9/9 [==============================] - 0s 55ms/step - loss: 2.7365 - mse: 2.7365
Epoch 3/100
9/9 [==============================] - 1s 57ms/step - loss: 2.7409 - mse: 2.7409
Epoch 4/100
9/9 [==============================] - 1s 64ms/step - loss: 4.5086 - mse: 4.5086
Epoch 5/100
9/9 [==============================] - 0s 52ms/step - loss: 1.6663 - mse: 1.6663
Epoch 6/100
9/9 [==============================] - 1s 63ms/step - loss: 1.9223 - mse: 1.9223
Epoch 7/100
9/9 [==============================] - 1s 65ms/step - loss: 2.6338 - mse: 2.6338
Epoch 8/100
9/9 [==============================] - 1s 62ms/step - loss: 2.2477 - mse: 2.2477
Epoch 9/100
9/9 [==============================] - 1s 87ms/step - loss: 2.6739 - mse: 2.6739
Epoch 10/100
9/9 [==============================] - 1s 59ms/step - loss: 2.4759 - mse: 2.4759
Epoch 11/100
9/9 [==============================] - 1s 98ms/step - 

[0.57341068983078, 0.57341068983078]

In [109]:
model = seq_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 2s 41ms/step - loss: 19.1531 - mse: 19.1531
Epoch 2/100
9/9 [==============================] - 0s 42ms/step - loss: 3.7305 - mse: 3.7305
Epoch 3/100
9/9 [==============================] - 0s 42ms/step - loss: 2.0354 - mse: 2.0354
Epoch 4/100
9/9 [==============================] - 0s 43ms/step - loss: 3.2770 - mse: 3.2770
Epoch 5/100
9/9 [==============================] - 0s 42ms/step - loss: 3.1778 - mse: 3.1778
Epoch 6/100
9/9 [==============================] - 0s 41ms/step - loss: 2.2695 - mse: 2.2695
Epoch 7/100
9/9 [==============================] - 0s 41ms/step - loss: 3.4121 - mse: 3.4121
Epoch 8/100
9/9 [==============================] - 0s 40ms/step - loss: 3.0558 - mse: 3.0558
Epoch 9/100
9/9 [==============================] - 0s 42ms/step - loss: 2.1883 - mse: 2.1883
Epoch 10/100
9/9 [==============================] - 0s 42ms/step - loss: 2.9685 - mse: 2.9685
Epoch 11/100
9/9 [==============================] - 0s 41ms/step - 

[0.6208373308181763, 0.6208373308181763]

In [126]:
## bi is worse than normal performance
model = bi_seq_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 5s 84ms/step - loss: 6.1043 - mse: 6.1043
Epoch 2/100
9/9 [==============================] - 1s 79ms/step - loss: 3.8328 - mse: 3.8328
Epoch 3/100
9/9 [==============================] - 1s 79ms/step - loss: 2.5214 - mse: 2.5214
Epoch 4/100
9/9 [==============================] - 1s 80ms/step - loss: 2.9117 - mse: 2.9117
Epoch 5/100
9/9 [==============================] - 1s 82ms/step - loss: 3.0211 - mse: 3.0211
Epoch 6/100
9/9 [==============================] - 1s 79ms/step - loss: 2.8933 - mse: 2.8933
Epoch 7/100
9/9 [==============================] - 1s 81ms/step - loss: 3.6974 - mse: 3.6974
Epoch 8/100
9/9 [==============================] - 1s 86ms/step - loss: 1.8647 - mse: 1.8647
Epoch 9/100
9/9 [==============================] - 1s 96ms/step - loss: 2.6132 - mse: 2.6132
Epoch 10/100
9/9 [==============================] - 1s 107ms/step - loss: 2.7378 - mse: 2.7378
Epoch 11/100
9/9 [==============================] - 1s 165ms/step - 

[0.8556870222091675, 0.8556870222091675]

In [110]:
model = stack_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 54ms/step - loss: 17.6332 - mse: 17.6332
Epoch 2/100
9/9 [==============================] - 1s 63ms/step - loss: 2.6510 - mse: 2.6510
Epoch 3/100
9/9 [==============================] - 1s 79ms/step - loss: 5.0003 - mse: 5.0003
Epoch 4/100
9/9 [==============================] - 1s 78ms/step - loss: 2.5484 - mse: 2.5484
Epoch 5/100
9/9 [==============================] - 0s 51ms/step - loss: 2.2173 - mse: 2.2173
Epoch 6/100
9/9 [==============================] - 0s 51ms/step - loss: 3.5303 - mse: 3.5303
Epoch 7/100
9/9 [==============================] - 1s 56ms/step - loss: 2.3582 - mse: 2.3582
Epoch 8/100
9/9 [==============================] - 0s 56ms/step - loss: 3.0422 - mse: 3.0422
Epoch 9/100
9/9 [==============================] - 1s 62ms/step - loss: 3.0257 - mse: 3.0257
Epoch 10/100
9/9 [==============================] - 0s 51ms/step - loss: 2.0737 - mse: 2.0737
Epoch 11/100
9/9 [==============================] - 0s 55ms/step - 

[0.5035983920097351, 0.5035983920097351]

In [129]:
model = bi_stack_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 14s 258ms/step - loss: 8.5756 - mse: 8.5756
Epoch 2/100
9/9 [==============================] - 2s 214ms/step - loss: 2.4571 - mse: 2.4571
Epoch 3/100
9/9 [==============================] - 2s 174ms/step - loss: 3.3495 - mse: 3.3495
Epoch 4/100
9/9 [==============================] - 2s 276ms/step - loss: 2.1248 - mse: 2.1248
Epoch 5/100
9/9 [==============================] - 2s 206ms/step - loss: 2.4043 - mse: 2.4043
Epoch 6/100
9/9 [==============================] - 2s 227ms/step - loss: 2.7888 - mse: 2.7888
Epoch 7/100
9/9 [==============================] - 2s 197ms/step - loss: 2.7376 - mse: 2.7376
Epoch 8/100
9/9 [==============================] - 2s 181ms/step - loss: 3.1392 - mse: 3.1392
Epoch 9/100
9/9 [==============================] - 2s 227ms/step - loss: 1.7792 - mse: 1.7792
Epoch 10/100
9/9 [==============================] - 2s 274ms/step - loss: 3.3027 - mse: 3.3027
Epoch 11/100
9/9 [==============================] - 2s 208

[0.7123511433601379, 0.7123511433601379]

In [ ]:
model = bi_back_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 4s 89ms/step - loss: 22.4418 - mse: 22.4418
Epoch 2/100
9/9 [==============================] - 1s 77ms/step - loss: 4.5435 - mse: 4.5435
Epoch 3/100
9/9 [==============================] - 1s 80ms/step - loss: 3.2138 - mse: 3.2138
Epoch 4/100
3/9 [=========>....................] - ETA: 0s - loss: 1.8505 - mse: 1.8505

In [ ]:
model = bi_back_stack_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

In [111]:
model = stack_3_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 4s 59ms/step - loss: 21.3409 - mse: 21.3409
Epoch 2/100
9/9 [==============================] - 1s 59ms/step - loss: 2.6011 - mse: 2.6011
Epoch 3/100
9/9 [==============================] - 1s 59ms/step - loss: 3.7139 - mse: 3.7139: 0s - loss: 4.4127 - mse: 4.4
Epoch 4/100
9/9 [==============================] - 1s 54ms/step - loss: 2.9117 - mse: 2.9117
Epoch 5/100
9/9 [==============================] - 1s 60ms/step - loss: 3.3292 - mse: 3.3292
Epoch 6/100
9/9 [==============================] - 1s 59ms/step - loss: 2.5813 - mse: 2.5813
Epoch 7/100
9/9 [==============================] - 1s 60ms/step - loss: 2.9778 - mse: 2.9778
Epoch 8/100
9/9 [==============================] - 1s 59ms/step - loss: 1.9670 - mse: 1.9670
Epoch 9/100
9/9 [==============================] - 1s 60ms/step - loss: 2.8510 - mse: 2.8510
Epoch 10/100
9/9 [==============================] - 1s 62ms/step - loss: 2.7390 - mse: 2.7390
Epoch 11/100
9/9 [===================

[0.5917391777038574, 0.5917391777038574]

In [112]:
model = deep_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 91ms/step - loss: 27.4856 - mse: 27.4856
Epoch 2/100
9/9 [==============================] - 1s 86ms/step - loss: 14.2364 - mse: 14.2364
Epoch 3/100
9/9 [==============================] - 1s 65ms/step - loss: 9.7113 - mse: 9.7113
Epoch 4/100
9/9 [==============================] - 1s 82ms/step - loss: 10.8618 - mse: 10.8618
Epoch 5/100
9/9 [==============================] - 1s 93ms/step - loss: 9.5097 - mse: 9.5097
Epoch 6/100
9/9 [==============================] - 1s 69ms/step - loss: 9.3194 - mse: 9.3194
Epoch 7/100
9/9 [==============================] - 1s 83ms/step - loss: 8.4292 - mse: 8.4292
Epoch 8/100
9/9 [==============================] - 1s 90ms/step - loss: 8.9412 - mse: 8.9412
Epoch 9/100
9/9 [==============================] - 1s 107ms/step - loss: 8.4190 - mse: 8.4190
Epoch 10/100
9/9 [==============================] - 1s 86ms/step - loss: 8.9947 - mse: 8.9947
Epoch 11/100
9/9 [==============================] - 1s 69ms/st

[1.120126485824585, 1.120126485824585]

In [113]:
model = lstm_dense_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 76ms/step - loss: 12.8647 - mse: 12.8647
Epoch 2/100
9/9 [==============================] - 1s 76ms/step - loss: 1.3657 - mse: 1.3657
Epoch 3/100
9/9 [==============================] - 1s 55ms/step - loss: 3.3670 - mse: 3.3670
Epoch 4/100
9/9 [==============================] - 1s 99ms/step - loss: 3.1556 - mse: 3.1556
Epoch 5/100
9/9 [==============================] - 1s 73ms/step - loss: 3.6876 - mse: 3.6876
Epoch 6/100
9/9 [==============================] - 1s 72ms/step - loss: 2.4851 - mse: 2.4851
Epoch 7/100
9/9 [==============================] - 1s 66ms/step - loss: 2.3636 - mse: 2.3636
Epoch 8/100
9/9 [==============================] - 1s 81ms/step - loss: 3.7620 - mse: 3.7620
Epoch 9/100
9/9 [==============================] - 1s 61ms/step - loss: 2.6443 - mse: 2.6443
Epoch 10/100
9/9 [==============================] - 1s 81ms/step - loss: 3.1290 - mse: 3.1290
Epoch 11/100
9/9 [==============================] - 1s 99ms/step - 

[1.642591118812561, 1.642591118812561]

In [114]:
model = batch_norm_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 4s 89ms/step - loss: 32.2926 - mse: 32.2926
Epoch 2/100
9/9 [==============================] - 1s 71ms/step - loss: 16.4980 - mse: 16.4980
Epoch 3/100
9/9 [==============================] - 1s 65ms/step - loss: 10.7462 - mse: 10.7462
Epoch 4/100
9/9 [==============================] - 1s 72ms/step - loss: 7.6862 - mse: 7.6862
Epoch 5/100
9/9 [==============================] - 1s 62ms/step - loss: 9.0544 - mse: 9.0544
Epoch 6/100
9/9 [==============================] - 1s 62ms/step - loss: 12.3728 - mse: 12.3728
Epoch 7/100
9/9 [==============================] - 1s 64ms/step - loss: 6.2003 - mse: 6.2003
Epoch 8/100
9/9 [==============================] - 1s 62ms/step - loss: 3.1740 - mse: 3.1740
Epoch 9/100
9/9 [==============================] - 1s 92ms/step - loss: 3.8392 - mse: 3.8392
Epoch 10/100
9/9 [==============================] - 1s 59ms/step - loss: 3.5257 - mse: 3.5257
Epoch 11/100
9/9 [==============================] - 1s 62ms/s

[5.653958320617676, 5.653958320617676]

In [115]:
model = layer_norm_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 76ms/step - loss: 17.6656 - mse: 17.6656
Epoch 2/100
9/9 [==============================] - 1s 91ms/step - loss: 3.3521 - mse: 3.3521
Epoch 3/100
9/9 [==============================] - 1s 66ms/step - loss: 2.6646 - mse: 2.6646
Epoch 4/100
9/9 [==============================] - 1s 59ms/step - loss: 3.2158 - mse: 3.2158
Epoch 5/100
9/9 [==============================] - 1s 66ms/step - loss: 2.9120 - mse: 2.9120
Epoch 6/100
9/9 [==============================] - 1s 54ms/step - loss: 2.1421 - mse: 2.1421
Epoch 7/100
9/9 [==============================] - 1s 61ms/step - loss: 2.1811 - mse: 2.1811
Epoch 8/100
9/9 [==============================] - 1s 71ms/step - loss: 3.2929 - mse: 3.2929
Epoch 9/100
9/9 [==============================] - 1s 73ms/step - loss: 2.7245 - mse: 2.7245
Epoch 10/100
9/9 [==============================] - 1s 68ms/step - loss: 1.7782 - mse: 1.7782
Epoch 11/100
9/9 [==============================] - 1s 87ms/step - 

[0.7952393293380737, 0.7952393889427185]

In [116]:
## much better with drop out than without. 
model = batch_norm_drop_lstm_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 58ms/step - loss: 37.4272 - mse: 37.4272
Epoch 2/100
9/9 [==============================] - 1s 53ms/step - loss: 19.7787 - mse: 19.7787
Epoch 3/100
9/9 [==============================] - 0s 53ms/step - loss: 12.8521 - mse: 12.8521
Epoch 4/100
9/9 [==============================] - 0s 51ms/step - loss: 9.2256 - mse: 9.2256
Epoch 5/100
9/9 [==============================] - 0s 51ms/step - loss: 8.3054 - mse: 8.3054
Epoch 6/100
9/9 [==============================] - 0s 49ms/step - loss: 9.1121 - mse: 9.1121
Epoch 7/100
9/9 [==============================] - 0s 53ms/step - loss: 8.0718 - mse: 8.0718
Epoch 8/100
9/9 [==============================] - 0s 51ms/step - loss: 7.7608 - mse: 7.7608
Epoch 9/100
9/9 [==============================] - 0s 52ms/step - loss: 6.9517 - mse: 6.9517
Epoch 10/100
9/9 [==============================] - 0s 52ms/step - loss: 6.5395 - mse: 6.5395
Epoch 11/100
9/9 [==============================] - 0s 51ms/ste

[3.886112689971924, 3.886112689971924]

In [108]:
model = gru_model()
model.fit(data_gen_train, epochs=100, callbacks=[early_stop])
model.summary()
model.evaluate(data_gen_test)

Epoch 1/100
9/9 [==============================] - 3s 58ms/step - loss: 5.1317 - mse: 5.1317
Epoch 2/100
9/9 [==============================] - 0s 37ms/step - loss: 3.1296 - mse: 3.1296
Epoch 3/100
9/9 [==============================] - 0s 34ms/step - loss: 2.4490 - mse: 2.4490
Epoch 4/100
9/9 [==============================] - 0s 33ms/step - loss: 3.5780 - mse: 3.5780
Epoch 5/100
9/9 [==============================] - 0s 39ms/step - loss: 2.1589 - mse: 2.1589
Epoch 6/100
9/9 [==============================] - 0s 54ms/step - loss: 1.9563 - mse: 1.9563
Epoch 7/100
9/9 [==============================] - 0s 33ms/step - loss: 2.2692 - mse: 2.2692
Epoch 8/100
9/9 [==============================] - 0s 50ms/step - loss: 2.6931 - mse: 2.6931
Epoch 9/100
9/9 [==============================] - 0s 35ms/step - loss: 3.2158 - mse: 3.2158
Epoch 10/100
9/9 [==============================] - 0s 42ms/step - loss: 2.3302 - mse: 2.3302
Epoch 11/100
9/9 [==============================] - 0s 41ms/step - lo

[0.7834493517875671, 0.7834493517875671]